In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017D0A79F2D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017D0A78CE90>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Bharat and I am a AI Engineer")])

AIMessage(content="Hi Bharat!\n\nIt's nice to meet you.  As an AI, I don't have a name, but you can call me Gemma.\n\nWhat kind of AI engineering work do you do?  I'm always interested in learning more about what people are building with AI.\n", response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 26, 'total_tokens': 89, 'completion_time': 0.126732351, 'prompt_time': 0.004296507, 'queue_time': None, 'total_time': 0.131028858}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8dca5bfc-477b-438e-a7a6-4a25ca040695-0', usage_metadata={'input_tokens': 26, 'output_tokens': 63, 'total_tokens': 89})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Bharat and I am a AI Engineer"),
        AIMessage(content="Hi Bharat, it's nice to meet you!  \n\nAs an AI myself, I'm always interested in connecting with other AI professionals.  What kind of AI engineering work do you do?  \n\nAre you working on any cool projects you'd like to tell me about?\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Bharat and that you are an AI Engineer!  😊  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat. \n", response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 108, 'total_tokens': 155, 'completion_time': 0.094916507, 'prompt_time': 0.017504569, 'queue_time': None, 'total_time': 0.112421076}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba094826-d900-4175-ad5b-b41088710982-0', usage_metadata={'input_tokens': 108, 'output_tokens': 47, 'total_tokens': 155})

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Bharat and I am an AI Engineer")],
    config=config
)

In [8]:
response.content

"Hi Bharat, it's nice to meet you!\n\nAs an AI, I don't have personal experiences or feelings, but I'm always eager to learn more about what people do. \n\nWhat kind of AI engineering work are you involved in?  \n\nI'm curious to hear about your projects and the challenges you face.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Bharat.  You told me at the beginning of our conversation! 😊  \n\nIs there anything else I can help you with? \n', response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 113, 'total_tokens': 146, 'completion_time': 0.066, 'prompt_time': 0.010346715, 'queue_time': None, 'total_time': 0.07634671500000001}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-72a9dee3-bb13-40d5-a5fa-20e40cf82fc7-0', usage_metadata={'input_tokens': 113, 'output_tokens': 33, 'total_tokens': 146})

In [10]:
# change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.  \n\nWhat can I do for you today?\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John. 😊  \n\nI remember!  Is there anything else I can help you with?\n'

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Bharat")]})

AIMessage(content="Hi Bharat, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😄  \n\n", response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 35, 'total_tokens': 63, 'completion_time': 0.058008081, 'prompt_time': 0.002542613, 'queue_time': None, 'total_time': 0.060550694}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4b1bc74e-b862-4806-86dc-e051ce7c59fc-0', usage_metadata={'input_tokens': 35, 'output_tokens': 28, 'total_tokens': 63})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Bharat")],
    config=config
)

response

AIMessage(content="Hello Bharat!  \n\nIt's great to meet you. What can I do for you today?  😄  \n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 72, 'total_tokens': 99, 'completion_time': 0.054, 'prompt_time': 0.004411464, 'queue_time': None, 'total_time': 0.058411463999999996}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-07aa5640-9043-4a06-86e2-c86f9034e68d-0', usage_metadata={'input_tokens': 72, 'output_tokens': 27, 'total_tokens': 99})

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bharat.  😊  \n\nIs there anything else I can help you with?\n'

In [19]:
# Let's add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [22]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Bharat")],"language":"Telugu"})
response.content

'నమస్కారం, భారత్! 😊 \n\nనేను మీకు సహాయం చేయగలను. ఏదైనా అడిగితే చెప్పండి. \n'

In [23]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [25]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Bharath")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते भारथ! \n\nमुझे आपसे मिलकर खुशी हो रही है। \n\nआप मुझे कैसे मदद कर सकते हैं? \n'

In [28]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Tamil"},
    config=config,
)
response.content

'உங்கள் பெயர் பாரத்.  \n'

In [29]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

d:\ProjectsDS\LCEL\venv-lcel1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\ProjectsDS\LCEL\venv-lcel1\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sbkva\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this artic

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [31]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a helpful assistant, I don't have access to your personal information, including your favorite ice cream flavor! \n\nWhat's your favorite ice cream flavor? 😊🍦\n"

In [32]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2".  \n\n\n\nLet me know if I can help with any other math problems! 😊  \n'

In [33]:
# Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [34]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"

In [35]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant with no memory of past conversations, I don't know what math problem you asked.\n\nIf you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"